Pull Data from FRED and other sources

In [1]:
# Import Dependencies and API Key

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import os
import json

# Import API key. Git ignore to be used in the future to hide FRED API key, but isn't utilized yet
from api_keys import fred_key

Extract Economic Data

In [169]:
#### Dictionary containing the FRED code to pull data from the API as the keys, and the name of the corresponsing main series
#### as named in my database as the values.

data_extract_dict = {'M1SL':'m1',
                      'M2SL':'m2',
                      'NONM1':'non_m1_components_m2',
                      'M1V':'m1v',
                      'M2V':'m2v',
                      'BUSLOANS':'commercial_industrial_loans',
                      'FEDFUNDS':'fed_funds_rate',
                      'DPSACBW027SBOG':'commercial_bank_deposits',
                      'TLAACBW027SBOG':'commercial_bank_assets',
                      'TOTRESNS':'reserve_balances',
                      'TOTBKCR':'commercial_bank_credit',
                      'MORTGAGE30US':'_30yr_fixed_rate_mortgage',
                      'CONSUMER':'consumer_loans_com_banks',
                      'CASACBW027SBOG':'commercial_bank_cash_assets',
                      'POPTHM':'pop',
                      'PCEPI':'pce_index',
                      'UNRATE':'unemployment',
                      'PSAVERT':'personal_savings_rate',
                      'CES0500000003':'average_hourly_wage',
                      'PMSAVE':'personal_savings',
                      'CUUR0000SETA01':'cpi_vehicles',
                      'APU0000708111':'cpi_eggs',
                      'CPIAPPSL':'cpi_apparel_cities',
                      'CPIHOSNS':'cpi_housing_cities',
                      'PCEDGC96':'real_pce_durable_goods',
                      'CPITRNSL':'cpi_urban_transportation',
                      'PCE':'pce',
                      'CIVPART':'labor_participation_rate',
                      'PCEC96':'real_pce',
                      'PCEDG':'pce_durable_goods',
                      'JTSJOL':'job_openings_nonfarm',
                      'PCEND':'pce_nondurable_goods',
                      'DSPIC96':'real_disposable_personal_income',
                      'ECOMPCTSA':'ecommerce_pct_of_totalsales',
                      'MSPUS':'median_house_sale_price',
                      'HDTGPDUSQ163N':'house_debt_gdp_ratio',
                      'OPHNFB':'real_output_hour',
                      'RRVRUSQ156N':'rental_vacancy_rate',
                      'DRSFRMACBS':'mortgage_delinquency',
                      'TDSP':'household_debt_service_pmtpctgdp',
                      'RHORUSQ156N':'homeownership_rate',
                      'DRCCLACBS':'creditcard_delinquency_rate',
                      'WFRBLT01026':'wealth_share_top1pct',
                      'GPSAVE':'gross_private_saving',
                      'QUSR628BIS':'real_residential_property_price',
                      'WFRBLB50107':'bottom_50pct_net_worth',
                      'NCBCMDPMVCE':'debt_as_pct_corporate_equities',
                      'WFRBLT01026':'wealth_total_top1pct',
                      'DRCLACBS':'consumer_loan_delinquency_rate',
                      'CPIAUCSL':'cpi',
                      'MICH':'inflation_expectation',
                      'CPILFESL':'cpi_core',
                      'CPIMEDSL':'cpi_medical',
                      'CUUR0000SA0R':'cpi_urban',
                      'CPIFABSL':'cpi_food_bev',
                      'STLFSI2':'financial_stress',
                      'WALCL':'fed_assets',
                      'TREAST':'fed_res_held_treasuries',
                      'WTREGEN':'fed_liabilities_non_reserve_deposits',
                      'RESPPANWW':'total_fed_assets',
                      'BOPGSTB':'net_trade',
                      'IMPGSC1':'real_imports',
                      'IMPGS':'imports_goods_services',
                      'INDCPIALLMINMEI':'cpi_india',
                      'IMPCH':'imports_from_china',
                      'IR':'all_commodities_import_price_index',
                      'GDP':'gdp',
                      'A939RC0Q052SBEA':'nom_gdpcap',
                      'GDPC1':'real_gdp',
                      'A939RX0Q048SBEA':'real_gdpcap',
                      'GDPDEF':'gdp_deflator',
                      'GFDEBTN':'federal_debt',
                      'GFDEGDQ188S':'debt_pct_gdp',
                      'W068RCQ027SBEA':'government_expenditures',
                      'FYGFDPUN':'federal_debt_held_by_public',
                      'FDHBFRBN':'fr_held_debt',
                      'B087RC1Q027SBEA':'government_transfer_payments',
                      'M318501Q027NBEA':'federal_surplus_deficit',
                      'B075RC1Q027SBEA':'corporate_income_tax_receipts',
                      'TTLCONS':'construction_spending',
                      'HOUST':'housing_starts',
                      'GPDIC1':'real_gross_domestic_private_investment',
                      'FYFSD':'deficit_surplus',
                      'MEHOINUSA672N':'real_median_house_income',
                      'FPCPITOTLZGUSA':'inflation_consumer_price',
                      'USEPUINDXD':'economic_uncertainty',
                      'PPIACO':'ppi_all_commodities',
                      'WPU0911':'ppi_wood_pulp',
                      'WPU101707':'ppi_metals',
                      'PCU325211325211':'ppi_plastics_resins',
                      'WPU101':'ppi_iron_steel',
                      'PWHEAMTUSDQ':'global_wheat_price',
                      'WPU10170502':'ppi_steel_wire',
                      'PCU484121484121':'ppi_freight',
                      'PALUMUSDM':'global_aluminum_price',
                      'PCU44414441':'ppi_building_materials',
                      'WPU0811':'ppi_wood_lumber',
                      'PMAIZMTUSDM':'global_corn_price',
                      'PIORECRUSDM':'global_iron_price',
                      'PRUBBUSDM':'global_rubber_price',
                      'WPU081':'ppi_lumber',
                      'PCU32733273':'ppi_cement_concrete',
                      'PCU33443344':'ppi_semiconductors_electronics',
                      'UMCSENT':'consumer_sentiment',
                      'CP':'corporate_profits_after_tax',
                      'PCESV':'pce_services',
                      'CUUR0000SEHA':'cpi_primary_rent',
                      'WSHOMCB':'fed_mbs',
                      'NETEXP':'net_exports',
                      'A019RE1A156NBEA':'net_exports_pctofgdp',
                      'GNP':'gnp',
                      'GPDI':'gross_domestic_private_investment',
                      'DCOILWTICO':'price_per_barrel',
                      'T10YIE':'_10_year_breakeven_inflation',
                      'T5YIFR':'inf_expectation_5yr',
                      'PCOPPUSDM':'copper_price',
                      'PCUOMFGOMFG':'ppi_manufacturing',
                      'DDDM01USA156NWDB':'stock_market_cap'}

In [191]:
#### Defines main function for extracting and transforming data.  This function is used on series with no known calculation errors
#### The function extracts raw data from the API in JSON format, pulls dates and observed valuse from the JSON, storing them in 
#### lists, runs calculations on the stored values for the _change and _pct_change columns for each series, storing those calculated
#### values in additional lists, and appends all three lists that correspond with each series to the all_data dictionary.  Error
#### handling is included with the try/except language so that the entire function does not stop if there is an unexpected error
#### when extracting or transforming a single series.

# all_data = {}
def fred_extract(series_dict):
    for key, value in series_dict.items():
        try:
            data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={key}&api_key={fred_key}&file_type=json')
            series_json = data.json()
            series_json_obs=series_json['observations']
            series_dates = []
            series_values = []
            series_change_values = [0]
            series_pct_change_values = [0]
            change_index = 1
            for each_item in range(len(series_json_obs)):
                item_date=series_json_obs[each_item]['date']
                item_value=series_json_obs[each_item]['value']
                if item_value != ".":
                    series_dates.append(series_json_obs[each_item]['date'])
                    series_values.append(float(series_json_obs[each_item]['value']))
        except:
            print(f'Error extracting {key}')
        for each_value in range(len(series_values)-1):
            try:
                if (series_values[change_index] > series_values[each_value]):
                    if (series_values[each_value] > 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] < 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = abs(change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = 100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                elif (series_values[change_index] < series_values[each_value]):
                    if (series_values[each_value] > 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (change/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] < 0):
                        change = series_values[change_index]-series_values[each_value]
                        pct_change = (abs(change)/series_values[each_value])*100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                    elif (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = -100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                elif (series_values[change_index] == series_values[each_value]):
                    change = 0
                    pct_change = 0
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (series_values[change_index] < series_values[each_value]):
                    if (series_values[each_value] == 0):
                        change = series_values[change_index]
                        pct_change = -100
                        series_change_values.append(change)
                        series_pct_change_values.append(pct_change)
                change_index = change_index + 1
                all_data[value]=[series_dates,series_values,series_change_values,series_pct_change_values]
            except:
                print(f'Error running calculations on {value}') 
            

In [197]:
#### Executes fred_extract function defined above.

fred_extract(data_extract_dict)

Error extracting GOLDAMGBD228NLBM


UnboundLocalError: local variable 'series_values' referenced before assignment

In [274]:
#### Creates 14 Pandas DataFrames that correspond with the first main series that appear in each of my database tables.
#### The loop in the cell below creates individual DataFrames for each remaining series and merges them with these 14

banks_week_month_df = pd.DataFrame(all_data['commercial_industrial_loans']).transpose().rename(columns={0:"Date",1:"commercial_industrial_loans",2:"commercial_industrial_loans_change",3:"commercial_industrial_loans_pct_change"}).set_index(['Date'])
m1m2_df = pd.DataFrame(all_data['m1']).transpose().rename(columns={0:"Date",1:"m1",2:"m1_change",3:"m1_pct_change"}).set_index(['Date'])
consumer_monthly_df = pd.DataFrame(all_data['pop']).transpose().rename(columns={0:"Date",1:"pop",2:"pop_change",3:"pop_pct_change"}).set_index(['Date'])
consumers_quarterly_df = pd.DataFrame(all_data['ecommerce_pct_of_totalsales']).transpose().rename(columns={0:"Date",1:"ecommerce_pct_of_totalsales",2:"ecommerce_pct_of_totalsales_change",3:"ecommerce_pct_of_totalsales_pct_change"}).set_index(['Date'])
cpi_monthly_df = pd.DataFrame(all_data['cpi']).transpose().rename(columns={0:"Date",1:"cpi",2:"cpi_change",3:"cpi_pct_change"}).set_index(['Date'])
federal_reserve_weekly_df = pd.DataFrame(all_data['financial_stress']).transpose().rename(columns={0:"Date",1:"financial_stress",2:"financial_stress_change",3:"financial_stress_pct_change"}).set_index(['Date'])
foreign_trade_month_quarter_df = pd.DataFrame(all_data['net_trade']).transpose().rename(columns={0:"Date",1:"net_trade",2:"net_trade_change",3:"net_trade_pct_change"}).set_index(['Date'])
gdp_quarterly_df = pd.DataFrame(all_data['gdp']).transpose().rename(columns={0:"Date",1:"gdp",2:"gdp_change",3:"gdp_pct_change"}).set_index(['Date'])
government_quarterly_df = pd.DataFrame(all_data['federal_debt']).transpose().rename(columns={0:"Date",1:"federal_debt",2:"federal_debt_change",3:"federal_debt_pct_change"}).set_index(['Date'])
investment_month_quarter_df = pd.DataFrame(all_data['construction_spending']).transpose().rename(columns={0:"Date",1:"construction_spending",2:"construction_spending_change",3:"construction_spending_pct_change"}).set_index(['Date'])
misc_annual_df = pd.DataFrame(all_data['deficit_surplus']).transpose().rename(columns={0:"Date",1:"deficit_surplus",2:"deficit_surplus_change",3:"deficit_surplus_pct_change"}).set_index(['Date'])
misc_daily_df = pd.DataFrame(all_data['inf_expectation_5yr']).transpose().rename(columns={0:"Date",1:"inf_expectation_5yr",2:"inf_expectation_5yr_change",3:"inf_expectation_5yr_pct_change"}).set_index(['Date'])
ppi_monthly_df = pd.DataFrame(all_data['ppi_manufacturing']).transpose().rename(columns={0:"Date",1:"ppi_manufacturing",2:"ppi_manufacturing_change",3:"ppi_manufacturing_pct_change"}).set_index(['Date'])
velocity_df = pd.DataFrame(all_data['m1v']).transpose().rename(columns={0:"Date",1:"m1v",2:"m1v_change",3:"m1v_pct_change"}).set_index(['Date'])

In [275]:
#### Lists of the main series columns for each table.  Additionally, for each of these series, "_change" and "pct_change" are 
#### calculated in the extraction and transformation functions.  These lists serve to pull specific series from the all_data 
#### dictionary where all data extracted from the FRED API is stored.

m1m2_column_list = ['m2','non_m1_components_m2']
velocity_column_list = ['m2v']
banks_week_month_column_list = ['fed_funds_rate','commercial_bank_deposits','commercial_bank_assets',\
                               'reserve_balances','commercial_bank_credit','_30yr_fixed_rate_mortgage','consumer_loans_com_banks',\
                               'commercial_bank_cash_assets']
consumer_monthly_column_list = ['pce_index','unemployment','personal_savings_rate','average_hourly_wage','personal_savings',\
                               'consumer_sentiment','cpi_vehicles','cpi_eggs','cpi_apparel_cities','cpi_housing_cities',\
                               'real_pce_durable_goods','cpi_urban_transportation','pce','labor_participation_rate','real_pce',\
                               'pce_durable_goods','job_openings_nonfarm','pce_nondurable_goods','real_disposable_personal_income']
consumers_quarterly_column_list = ['median_house_sale_price','house_debt_gdp_ratio','real_output_hour',\
                                  'corporate_profits_after_tax','pce_services','rental_vacancy_rate','mortgage_delinquency',\
                                  'household_debt_service_pmtpctgdp','homeownership_rate','creditcard_delinquency_rate','wealth_share_top1pct',\
                                  'gross_private_saving','real_residential_property_price','bottom_50pct_net_worth',\
                                  'debt_as_pct_corporate_equities','wealth_total_top1pct','consumer_loan_delinquency_rate']
cpi_monthly_column_list = ['inflation_expectation','cpi_core','cpi_medical','cpi_urban','cpi_primary_rent','cpi_food_bev']
federal_reserve_weekly_column_list = ['fed_assets','fed_res_held_treasuries','fed_liabilities_non_reserve_deposits',\
                                     'fed_mbs','total_fed_assets']
foreign_trade_month_quarter_column_list = ['net_exports','net_exports_pctofgdp','real_imports','imports_goods_services',\
                                          'cpi_india','imports_from_china','all_commodities_import_price_index']
gdp_quarterly_column_list = ['nom_gdpcap','real_gdp','real_gdpcap','gdp_deflator','gnp']
government_quarterly_column_list = ['debt_pct_gdp','government_expenditures','federal_debt_held_by_public','fr_held_debt',\
                                   'government_transfer_payments','federal_surplus_deficit','corporate_income_tax_receipts']
investment_month_quarter_column_list = ['housing_starts','real_gross_domestic_private_investment',\
                                       'gross_domestic_private_investment']
misc_annual_column_list = ['stock_market_cap','real_median_house_income','inflation_consumer_price']
misc_daily_column_list = ['price_per_barrel','economic_uncertainty','_10_year_breakeven_inflation']
ppi_monthly_column_list = ['ppi_all_commodities','ppi_wood_pulp','ppi_metals','copper_price','ppi_plastics_resins',\
                          'ppi_iron_steel','global_wheat_price','ppi_steel_wire','ppi_freight','global_aluminum_price',\
                          'ppi_building_materials','ppi_wood_lumber','global_corn_price','global_iron_price','global_rubber_price',\
                          'ppi_lumber','ppi_cement_concrete','ppi_semiconductors_electronics']
#stocks_gold_daily_column_list = ['djia_close','nasdaq_close','sp500_close','gold_price']
# all_table_column_dict = {m1m2_df:m1m2_column_list,velocity_df:velocity_column_list,banks_week_month_df:banks_week_month_column_list,\
#                          consumer_monthly_df:consumer_monthly_column_list,consumers_quarterly_df:consumers_quarterly_column_list,\
#                          cpi_monthly_df:cpi_monthly_column_list,federal_reserve_weekly_df:federal_reserve_weekly_column_list,\
#                          foreign_trade_month_quarter_df:foreign_trade_month_quarter_column_list,gdp_quarterly_df:gdp_quarterly_column_list,\
#                          government_quarterly_df:government_quarterly_column_list,investment_month_quarter_df:investment_month_quarter_column_list,\
#                          misc_annual_df:misc_annual_column_list,misc_daily_df:misc_daily_column_list,ppi_monthly_df:ppi_monthly_column_list}
# all_table_column_list = [m1m2_column_list, velocity_column_list]
# all_table_df_list = [m1m2_df,velocity_df]

In [276]:
for column in m1m2_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    m1m2_df = m1m2_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in velocity_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    velocity_df = velocity_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in banks_week_month_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    banks_week_month_df = banks_week_month_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in consumer_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    consumer_monthly_df = consumer_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in consumers_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    consumers_quarterly_df = consumers_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in cpi_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    cpi_monthly_df = cpi_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in federal_reserve_weekly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    federal_reserve_weekly_df = federal_reserve_weekly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in foreign_trade_month_quarter_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    foreign_trade_month_quarter_df = foreign_trade_month_quarter_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in gdp_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    gdp_quarterly_df = gdp_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in government_quarterly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    government_quarterly_df = government_quarterly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in investment_month_quarter_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    investment_month_quarter_df = investment_month_quarter_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in misc_annual_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    misc_annual_df = misc_annual_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in misc_daily_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    misc_daily_df = misc_daily_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])
for column in ppi_monthly_column_list:
    temp_df = pd.DataFrame(all_data[f'{column}']).transpose().rename(columns={0:'Date',1:f'{column}',2:f'{column}_change',3:f'{column}_pct_change'}).set_index(['Date'])
    ppi_monthly_df = ppi_monthly_df.merge(temp_df,how='outer',on='Date').sort_values(by=['Date'])

In [277]:
#### Import data manually pulled from WSJ (to be automated with web scraping at a later date), run change and pct_change calculations
#### organize into individual DataFrames, merge DataFrames.  Gold must be cleaned separately before merging
djia_path = os.path.join("../Manual Downloads/djia.csv")
nasdaq_path = os.path.join("../Manual Downloads/nasdaq.csv")
gold_path=os.path.join("../Manual Downloads/gold.csv")
sp500_path=os.path.join("../Manual Downloads/sp500.csv")
djia_data = pd.read_csv(djia_path)
nasdaq_data = pd.read_csv(nasdaq_path)
gold_data=pd.read_csv(gold_path)
sp500_data=pd.read_csv(sp500_path)
sp500_df=sp500_data.rename(columns={' Close':'sp500_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
sp500_df=sp500_df[::-1]
sp500_values = sp500_df['sp500_close'].tolist()
sp500_close_change = [0]
sp500_close_pct_change =[0]
change_index = 1
for i in range(len(sp500_values)-1):
    change = sp500_values[change_index] - sp500_values[i]
    pct_change = (change/sp500_values[i])*100
    sp500_close_change.append(change)
    sp500_close_pct_change.append(pct_change)
    change_index=change_index + 1
sp500_df['sp500_close_change']=sp500_close_change
sp500_df['sp500_close_pct_change']=sp500_close_pct_change
djia_df=djia_data.rename(columns={' Close':'djia_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
djia_df=djia_df[::-1]
djia_values = djia_df['djia_close'].tolist()
djia_close_change = [0]
djia_close_pct_change =[0]
change_index = 1
for j in range(len(djia_values)-1):
    change = djia_values[change_index] - djia_values[j]
    pct_change = (change/djia_values[i])*100
    djia_close_change.append(change)
    djia_close_pct_change.append(pct_change)
    change_index=change_index + 1
djia_df['djia_close_change']=djia_close_change
djia_df['djia_close_pct_change']=djia_close_pct_change
nasdaq_df=nasdaq_data.rename(columns={' Close':'nasdaq_close'}).drop(columns=[' Open',' High',' Low']).set_index('Date')
nasdaq_df=nasdaq_df[::-1]
nasdaq_values = nasdaq_df['nasdaq_close'].tolist()
nasdaq_close_change = [0]
nasdaq_close_pct_change =[0]
change_index = 1
for q in range(len(nasdaq_values)-1):
    change = nasdaq_values[change_index] - nasdaq_values[q]
    pct_change = (change/nasdaq_values[i])*100
    nasdaq_close_change.append(change)
    nasdaq_close_pct_change.append(pct_change)
    change_index=change_index + 1
nasdaq_df['nasdaq_close_change']=nasdaq_close_change
nasdaq_df['nasdaq_close_pct_change']=nasdaq_close_pct_change
gold_df=gold_data.rename(columns={'DATE':'Date','GOLDPMGBD228NLBM':'gold_price'}).set_index('Date')
gold_df=gold_df.loc[gold_df.loc[:,'gold_price']!='.',:]
gold_values = gold_df['gold_price'].tolist()
gold_price_change = [0]
gold_price_pct_change =[0]
change_index = 1
for g in range(len(gold_values)-1):
    change = float(gold_values[change_index]) - float(gold_values[g])
    pct_change = (change/float(gold_values[g]))*100
    gold_price_change.append(change)
    gold_price_pct_change.append(pct_change)
    change_index=change_index + 1
gold_df['gold_price_change']=gold_price_change
gold_df['gold_price_pct_change']=gold_price_pct_change
stocks_gold_daily_df = pd.merge(djia_df,nasdaq_df, how='outer',on='Date')
stocks_gold_daily_df = stocks_gold_daily_df.merge(sp500_df,how='outer',on='Date')
stocks_gold_daily_df = stocks_gold_daily_df.merge(gold_df,how='outer',on='Date').sort_values(by=['Date'])


In [278]:
#### Export all data to final .CSVs for upload to PostgreSQL
m1m2_df.to_csv('../DFs_for_DB/m1m2.csv')
velocity_df.to_csv('../DFs_for_DB/velocity.csv')
stocks_gold_daily_df.to_csv('../DFs_for_DB/stocks_gold_daily.csv')
banks_week_month_df.to_csv('../DFs_for_DB/banks_week_month.csv')
consumer_monthly_df.to_csv('../DFs_for_DB/consumer_monthly.csv')
consumers_quarterly_df.to_csv('../DFs_for_DB/consumers_quarterly.csv')
cpi_monthly_df.to_csv('../DFs_for_DB/cpi_monthly.csv')
federal_reserve_weekly_df.to_csv('../DFs_for_DB/federal_reserve_weekly.csv')
foreign_trade_month_quarter_df.to_csv('../DFs_for_DB/foreign_trade_month_quarter.csv')
gdp_quarterly_df.to_csv('../DFs_for_DB/gdp_quarterly.csv')
government_quarterly_df.to_csv('../DFs_for_DB/government_quarterly.csv')
investment_month_quarter_df.to_csv('../DFs_for_DB/investment_month_quarter.csv')
misc_annual_df.to_csv('../DFs_for_DB/misc_annual.csv')
misc_daily_df.to_csv('../DFs_for_DB/misc_daily.csv')
ppi_monthly_df.to_csv('../DFs_for_DB/ppi_monthly.csv')

AREA FOR TESTING OF FUNCTIONS FOR LAG, HANDLING OF DIFFERENT INTERVALS

In [369]:
reference_df = m1_df.merge(gdp_df, how="inner", on="Date")
reference_df

,Date,m1,gdp
0,1959-01-01,138.9,510.33
1,1959-04-01,139.7,522.653
2,1959-07-01,141.7,525.034
3,1959-10-01,140.5,528.6
4,1960-01-01,140,542.648
...,...,...,...
245,2020-04-01,4774.4,19477.4
246,2020-07-01,16773.8,21138.6
247,2020-10-01,17346.8,21477.6
248,2021-01-01,18100.6,22038.2


In [381]:
lag = 2
which_series = '_change'
which_series2 = '_change'

In [374]:
len(reference_df['m1'])

250

In [385]:
m1_gdp_df = reference_df
# series_change_values = [0]
# series_pct_change_values = [0]
columns_list = list(m1_gdp_df)
series1 = columns_list[1]
series2 = columns_list[2]
loop_values = [series1,series2]
for each_series_from_df in loop_values:
    new_change_index = 1
    series_change_values = [0]
    series_pct_change_values = [0]
    for each_value in range(len(m1_gdp_df[series1])-1):
        try:
            if (m1_gdp_df[each_series_from_df][new_change_index] > m1_gdp_df[each_series_from_df][each_value]):
                if (m1_gdp_df[each_series_from_df][each_value] > 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]-m1_gdp_df[each_series_from_df][each_value]
                    pct_change = (change/m1_gdp_df[each_series_from_df][each_value])*100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (m1_gdp_df[each_series_from_df][each_value] < 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]-m1_gdp_df[each_series_from_df][each_value]
                    pct_change = abs(change/m1_gdp_df[each_series_from_df][each_value])*100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (m1_gdp_df[each_series_from_df][each_value] == 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]
                    pct_change = 100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
            elif (m1_gdp_df[each_series_from_df][new_change_index] < m1_gdp_df[each_series_from_df][each_value]):
                if (m1_gdp_df[each_series_from_df][each_value] > 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]-m1_gdp_df[each_series_from_df][each_value]
                    pct_change = (change/m1_gdp_df[each_series_from_df][each_value])*100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (m1_gdp_df[each_series_from_df][each_value] < 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]-m1_gdp_df[each_series_from_df][each_value]
                    pct_change = (abs(change)/m1_gdp_df[each_series_from_df][each_value])*100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
                elif (m1_gdp_df[each_series_from_df][each_value] == 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]
                    pct_change = -100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
            elif (m1_gdp_df[each_series_from_df][new_change_index] == m1_gdp_df[each_series_from_df][each_value]):
                change = 0
                pct_change = 0
                series_change_values.append(change)
                series_pct_change_values.append(pct_change)
            elif (m1_gdp_df[each_series_from_df][new_change_index] < m1_gdp_df[each_series_from_df][each_value]):
                if (m1_gdp_df[each_series_from_df][each_value] == 0):
                    change = m1_gdp_df[each_series_from_df][new_change_index]
                    pct_change = -100
                    series_change_values.append(change)
                    series_pct_change_values.append(pct_change)
            new_change_index = new_change_index + 1
        except:
            print('something went wrong')
    m1_gdp_df[f'{each_series_from_df}_change'] = series_change_values
    m1_gdp_df[f'{each_series_from_df}_pct_change'] = series_pct_change_values
                    

In [386]:
m1_gdp_df = m1_gdp_df.loc[:,['Date',f'{series1}{which_series}',f'{series2}{which_series2}']]
m1_gdp_df                      

,Date,m1_change,gdp_change
0,1959-01-01,0.0,0.000
1,1959-04-01,0.8,12.323
2,1959-07-01,2.0,2.381
3,1959-10-01,-1.2,3.566
4,1960-01-01,-0.5,14.048
...,...,...,...
245,2020-04-01,755.2,-2003.923
246,2020-07-01,11999.4,1661.130
247,2020-10-01,573.0,339.023
248,2021-01-01,753.8,560.629


In [399]:
# pick up lag shuffle here
lagged_series = m1_gdp_df['m1_change'][1:-lag].tolist()
lagged_df = m1_gdp_df.iloc[lag+1:,:].loc[:,['Date','gdp_change']]
lagged_df['m1_change'] = lagged_series
lagged_df
# m1_changes = m1_change_df['m1_change']
# m1_changes = m1_changes[:-2]
# lagged_dict = {}
# changes_index = 0
# for each_date in m1_dates:
#     lagged_dict[each_date] = m1_changes[changes_index]
#     changes_index = changes_index +1
# lagged_dict
# Make dataframe

,Date,gdp_change,m1_change
3,1959-10-01,3.566,0.8
4,1960-01-01,14.048,2.0
5,1960-04-01,-1.568,-1.2
6,1960-07-01,4.524,-0.5
7,1960-10-01,-5.407,-0.4
...,...,...,...
245,2020-04-01,-2003.923,81.3
246,2020-07-01,1661.130,93.1
247,2020-10-01,339.023,755.2
248,2021-01-01,560.629,11999.4


In [400]:
lagged_series

[0.799999999999983,
 2.0,
 -1.1999999999999886,
 -0.5,
 -0.4000000000000057,
 0.5999999999999943,
 0.700000000000017,
 0.19999999999998863,
 1.0,
 0.8000000000000114,
 1.1999999999999886,
 1.0999999999999943,
 1.200000000000017,
 0.09999999999999432,
 0.19999999999998863,
 1.6000000000000227,
 1.3999999999999773,
 1.6000000000000227,
 1.299999999999983,
 1.0999999999999943,
 1.1000000000000227,
 2.0,
 2.3999999999999773,
 1.5,
 1.3000000000000114,
 1.0,
 3.0,
 3.0999999999999943,
 2.700000000000017,
 -1.5,
 0.8999999999999773,
 0.700000000000017,
 2.299999999999983,
 3.9000000000000057,
 3.5,
 2.700000000000017,
 2.299999999999983,
 3.9000000000000057,
 3.5,
 4.699999999999989,
 2.0,
 1.0,
 1.200000000000017,
 3.299999999999983,
 0.5,
 1.3000000000000114,
 4.900000000000006,
 2.5999999999999943,
 4.5,
 4.900000000000006,
 2.299999999999983,
 2.9000000000000057,
 5.5,
 3.200000000000017,
 6.199999999999989,
 6.5,
 1.1999999999999886,
 4.800000000000011,
 1.5,
 4.800000000000011,
 3.3999

In [388]:
gdp_df = pd.DataFrame(gdp_quarterly_df.loc[:,'gdp'])
m1_df = pd.DataFrame(m1m2_df.loc[:,'m1'])
m1_df = m1_df.reset_index()
gdp_df = gdp_df.reset_index()
m1_df

,Date,m1
0,1959-01-01,138.9
1,1959-02-01,139.4
2,1959-03-01,139.7
3,1959-04-01,139.7
4,1959-05-01,140.7
...,...,...
3074,2021-04-01,18915.7
3075,2021-05-01,19192.9
3076,2021-06-01,19238.2
3077,2021-07-01,19401.7


In [350]:
third_index = m1_change_df.iloc[3,:]
third_index

m1_change    0
Name: 1959-04-01, dtype: object

In [357]:
m1_dates = m1_change_df['Date']
m1_dates[0]

'1959-01-01'

In [358]:
m1_dates = m1_dates[2:]
m1_dates

2       1959-03-01
3       1959-04-01
4       1959-05-01
5       1959-06-01
6       1959-07-01
           ...    
3074    2021-04-01
3075    2021-05-01
3076    2021-06-01
3077    2021-07-01
3078    2021-08-01
Name: Date, Length: 3077, dtype: object

In [360]:
m1_changes = m1_change_df['m1_change']
m1_changes = m1_changes[:-2]
m1_changes

0           0
1         0.5
2         0.3
3           0
4           1
        ...  
3072    288.8
3073      280
3074    246.3
3075    277.2
3076     45.3
Name: m1_change, Length: 3077, dtype: object

In [363]:
lagged_dict = {}
changes_index = 0
for each_date in m1_dates:
    lagged_dict[each_date] = m1_changes[changes_index]
    changes_index = changes_index +1
lagged_dict

{'1959-03-01': 0,
 '1959-04-01': 0.5,
 '1959-05-01': 0.29999999999998295,
 '1959-06-01': 0,
 '1959-07-01': 1.0,
 '1959-08-01': 0.5,
 '1959-09-01': 0.5,
 '1959-10-01': 0.20000000000001705,
 '1959-11-01': -0.9000000000000057,
 '1959-12-01': -0.5,
 '1960-01-01': -0.09999999999999432,
 '1960-02-01': -0.5,
 '1960-03-01': 0.09999999999999432,
 '1960-04-01': -0.09999999999999432,
 '1960-05-01': -0.09999999999999432,
 '1960-06-01': -0.20000000000001705,
 '1960-07-01': 0,
 '1960-08-01': 0,
 '1960-09-01': 0.5999999999999943,
 '1960-10-01': 1.1000000000000227,
 '1960-11-01': -0.10000000000002274,
 '1960-12-01': -0.29999999999998295,
 '1961-01-01': 0,
 '1961-02-01': -0.20000000000001705,
 '1961-03-01': 0.4000000000000057,
 '1961-04-01': 0.5,
 '1961-05-01': 0.30000000000001137,
 '1961-06-01': 0.19999999999998863,
 '1961-07-01': 0.5999999999999943,
 '1961-08-01': 0.20000000000001705,
 '1961-09-01': 0,
 '1961-10-01': 0.5999999999999943,
 '1961-11-01': 0.30000000000001137,
 '1961-12-01': 0.29999999999

.


ALL CODE BELOW IS FROM WHEN I BUILT THIS PROJECT THE FIRST TIME.  IT WAS DONE HAPHAZARDLY AS MY SCOPE CONTINUED TO EXPAND AND AS I DISCOVERED ERRORS IN THE DATA THAT NEEDED CLEANING/FIXING.  IT IS ADMITTEDLY UGLY AND INEFFICIENT.


.

In [2]:
# data_series = ['M1SL','M2SL','MZM','M1V','M2V','CPIAUCSL','GFDEGDQ188S','FDHBFRBN','GFDEGDQ188S','GFDEBTN','FYFSD','FEDFUNDS'\
#                ,'GDPC1','GDP','A939RX0Q048SBEA','A939RC0Q052SBEA','POPTHM']

In [405]:
# url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_key}&file_type=json'
# url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json'
series_dicts = []
for series in data_series:
    data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
    series_json = data.json()
    series_json_obs=series_json['observations']
    series_dates = []
    series_values = []
    series = {}
    for each_item in range(len(series_json_obs)):
        series_dates.append(series_json_obs[each_item]['date'])
        series_values.append(series_json_obs[each_item]['value'])
    series_dict = dict(zip(series_dates,series_values))
    series_dicts.append(series_dict)

M1 - Create and format DF, Make Calculations

In [4]:

m1 = series_dicts[0]
m1_df = pd.DataFrame(m1.items())
m1_df = m1_df.rename(columns={0:"Date",1:"M1"})
m1_column_values = m1_df['M1'].tolist()
m1_monthly_change = [0]
m1_pct_change = [0]
m1_change_index = 1
for m in range(len(m1_column_values)-1):
    m1_monthly_change.append(float(m1_column_values[m1_change_index])-float(m1_column_values[m1_change_index-1]))
    m1_pct_change.append(((float(m1_column_values[m1_change_index])-float(m1_column_values[m1_change_index-1]))/
                                     float(m1_column_values[m1_change_index-1])*100))
    m1_change_index=m1_change_index+1
m1_df['M1_Change'] = m1_monthly_change
m1_df['M1_Pct_Change'] = m1_pct_change
m1_df = m1_df.set_index(['Date'])
# m1_df

M2 - Create and format DF, make calculations

In [5]:
m2 = series_dicts[1]
m2_df = pd.DataFrame(m2.items())
m2_df = m2_df.rename(columns={0:"Date",1:"M2"})
m2_column_values = m2_df['M2'].tolist()
m2_monthly_change = [0]
m2_pct_change = [0]
change_index = 1
for m in range(len(m2_column_values)-1):
    m2_monthly_change.append(float(m2_column_values[change_index])-float(m2_column_values[change_index-1]))
    m2_pct_change.append(((float(m2_column_values[change_index])-float(m2_column_values[change_index-1]))/
                                     float(m2_column_values[change_index-1])*100))
    change_index=change_index+1
m2_df['M2_Change'] = m2_monthly_change
m2_df['M2_Pct_Change'] = m2_pct_change
m2_df = m2_df.set_index(['Date'])
# m2_df

M1 Velocity - create and format DF, make calculations

In [6]:
m1v = series_dicts[3]
m1v_df = pd.DataFrame(m1v.items())
m1v_df = m1v_df.rename(columns={0:"Date",1:"M1V"})
m1v_column_values = m1v_df['M1V'].tolist()
m1v_monthly_change = [0]
m1v_pct_change = [0]
change_index = 1
for m in range(len(m1v_column_values)-1):
    m1v_monthly_change.append(float(m1v_column_values[change_index])-float(m1v_column_values[change_index-1]))
    m1v_pct_change.append(((float(m1v_column_values[change_index])-float(m1v_column_values[change_index-1]))/
                                     float(m1v_column_values[change_index-1])*100))
    change_index=change_index+1
m1v_df['M1V_Change'] = m1v_monthly_change
m1v_df['M1V_Pct_Change'] = m1v_pct_change
m1v_df = m1v_df.set_index(['Date'])
# m1v_df

M2 Velocity - create DF, make calculations

In [7]:
m2v = series_dicts[4]
m2v_df = pd.DataFrame(m2v.items())
m2v_df = m2v_df.rename(columns={0:"Date",1:"M2V"})
m2v_column_values = m2v_df['M2V'].tolist()
m2v_monthly_change = [0]
m2v_pct_change = [0]
change_index = 1
for m in range(len(m2v_column_values)-1):
    m2v_monthly_change.append(float(m2v_column_values[change_index])-float(m2v_column_values[change_index-1]))
    m2v_pct_change.append(((float(m2v_column_values[change_index])-float(m2v_column_values[change_index-1]))/
                                     float(m2v_column_values[change_index-1])*100))
    change_index=change_index+1
m2v_df['M2V_Change'] = m2v_monthly_change
m2v_df['M2V_Pct_Change'] = m2v_pct_change
m2v_df = m2v_df.set_index(['Date'])
# m2v_df

CPI - create df, make calculations

In [8]:
cpi = series_dicts[5]
cpi_df = pd.DataFrame(cpi.items())
cpi_df = cpi_df.rename(columns={0:"Date",1:"CPI"})
cpi_column_values = cpi_df['CPI'].tolist()
cpi_monthly_change = [0]
cpi_pct_change = [0]
change_index = 1
for m in range(len(cpi_column_values)-1):
    cpi_monthly_change.append(float(cpi_column_values[change_index])-float(cpi_column_values[change_index-1]))
    cpi_pct_change.append(((float(cpi_column_values[change_index])-float(cpi_column_values[change_index-1]))/
                                     float(cpi_column_values[change_index-1])*100))
    change_index=change_index+1
cpi_df['CPI_Change'] = cpi_monthly_change
cpi_df['CPI_Pct_Change'] = cpi_pct_change
cpi_df = cpi_df.set_index(['Date'])
# cpi_df

In [9]:
# calc = series_dicts[5]
# calc_df = pd.DataFrame(calc.items())
# calc_df = calc_df.rename(columns={0:"Date",1:"GDP"})
# calc_column_values = calc_df['GDP'].tolist()
# calc_monthly_change = [0]
# calc_pct_change = [0]
# change_index = 1
# for m in range(len(calc_column_values)-1):
#     calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
#     calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
#                                      float(calc_column_values[change_index-1])*100))
#     change_index=change_index+1
# calc_df['GDP_Change'] = calc_monthly_change
# calc_df['GDP_Pct_Change'] = calc_pct_change
# gdp_df = calc_df.set_index(['Date'])
# gdp_df

GDP - create DF, make calculations

In [10]:
calc = series_dicts[13]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"GDP"})
index_to_drop = [0,1,2,3]
calc_df = calc_df.drop(index=index_to_drop, axis=0)
calc_column_values = calc_df['GDP'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['GDP_Change'] = calc_monthly_change
calc_df['GDP_Pct_Change'] = calc_pct_change
gdp_df = calc_df.set_index(['Date'])
# gdp_df

Real GDP - create df, make calculations

In [11]:
calc = series_dicts[12]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Real_GDP"})
calc_column_values = calc_df['Real_GDP'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Real_GDP_Change'] = calc_monthly_change
calc_df['Real_GDP_Pct_Change'] = calc_pct_change
real_gdp_df = calc_df.set_index(['Date'])
# real_gdp_df

Population - create df, make calculations

In [12]:
calc = series_dicts[16]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Population"})
calc_column_values = calc_df['Population'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Pop_Change'] = calc_monthly_change
calc_df['Pop_Pct_Change'] = calc_pct_change
pop_df = calc_df.set_index(['Date'])
# pop_df

Federal Debt - create DF, make calculations

In [13]:
calc = series_dicts[9]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Federal_Debt"})
calc_column_values = calc_df['Federal_Debt'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Federal_Debt_Change'] = calc_monthly_change
calc_df['Federal_Debt_Pct_Change'] = calc_pct_change
debt_df = calc_df.set_index(['Date'])
# debt_df

Federal Funds Rate - create DF, make calculations

In [14]:
calc = series_dicts[11]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Fed_Funds_Rate"})
calc_column_values = calc_df['Fed_Funds_Rate'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
#     calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
#                                      float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Fed_Funds_Change'] = calc_monthly_change
# calc_df['GDP_Pct_Change'] = calc_pct_change
fedrate_df = calc_df.set_index(['Date'])
# fedrate_df

Debt Held By Fed Banks - Create DF, make calculations

In [15]:
calc = series_dicts[7]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"FR_Held_Debt"})
calc_column_values = calc_df['FR_Held_Debt'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['FR_held_debt_chg'] = calc_monthly_change
calc_df['FR_held_debt_pct_chg'] = calc_pct_change
frdebt_df = calc_df.set_index(['Date'])
# frdebt_df

Federal deficit - create DF, make calculations

In [16]:
calc = series_dicts[10]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Deficit_Surplus"})
calc_column_values = calc_df['Deficit_Surplus'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
#     calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
#                                      float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Deficit_Surplus_Change'] = calc_monthly_change
#calc_df['Def/Sur_Pct_Change'] = calc_pct_change
deficit_df = calc_df.set_index(['Date'])
# deficit_df

Debt as Percent of GDP - create df, make calculations

In [17]:
calc = series_dicts[6]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Debt_Pct_GDP"})
calc_column_values = calc_df['Debt_Pct_GDP'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
#     calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
#                                      float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Debt_Pct_GDP_Change'] = calc_monthly_change
#calc_df['GDP_Pct_Change'] = calc_pct_change
debtpct_df = calc_df.set_index(['Date'])
# debtpct_df

Real GDP per Capita - create DF, make calculations

In [18]:
calc = series_dicts[14]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Real_GDP/Capita"})
calc_column_values = calc_df['Real_GDP/Capita'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Real_GDP/Cap_Change'] = calc_monthly_change
calc_df['Real_GDP/Cap_Pct_Change'] = calc_pct_change
rgdpcap_df = calc_df.set_index(['Date'])
# rgdpcap_df

Nominal GDP per capita - create DF, make calculations

In [19]:
calc = series_dicts[15]
calc_df = pd.DataFrame(calc.items())
calc_df = calc_df.rename(columns={0:"Date",1:"Nom_GDP/Cap"})
calc_column_values = calc_df['Nom_GDP/Cap'].tolist()
calc_monthly_change = [0]
calc_pct_change = [0]
change_index = 1
for m in range(len(calc_column_values)-1):
    calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
    calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                     float(calc_column_values[change_index-1])*100))
    change_index=change_index+1
calc_df['Nom_GDP/Cap_Change'] = calc_monthly_change
calc_df['Nom_GDP/Cap_Pct_Change'] = calc_pct_change
ngdpcap_df = calc_df.set_index(['Date'])
# ngdpcap_df

In [20]:
####### Save all dataframes as csvs for backup - NOT NECESSARY
# ngdpcap_df.to_csv('calc_dataframes/NomGDPperCap.csv')
# rgdpcap_df.to_csv('calc_dataframes/RealGDPperCap.csv')
# debtpct_df.to_csv('calc_dataframes/DebtPctofGDP.csv')
# deficit_df.to_csv('calc_dataframes/FederalDeficit.csv')
# frdebt_df.to_csv('calc_dataframes/DebtHeldByFed.csv')
# fedrate_df.to_csv('calc_dataframes/FedFundsRate.csv')
# debt_df.to_csv('calc_dataframes/TotalFederalDebt.csv')
# pop_df.to_csv('calc_dataframes/Population.csv')
# real_gdp_df.to_csv('calc_dataframes/RealGDP.csv')
# gdp_df.to_csv('calc_dataframes/NomGDP.csv')
# cpi_df.to_csv('calc_dataframes/CPI.csv')
# m1v_df.to_csv('calc_dataframes/M1velocity.csv')
# m2v_df.to_csv('calc_dataframes/M2velocity.csv')
# m1_df.to_csv('calc_dataframes/M1.csv')
# m2_df.to_csv('calc_dataframes/M2.csv')

Extract next priority level of data

In [21]:
second_data_series = ['DDDM01USA156NWDB','MICH','WALCL','DCOILWTICO','CPILFESL','PPIACO','STLFSI2','CUUR0000SA0R','CPIMEDSL','BOPGSTB'\
                      ,'GNP']
later_not_working =['GOLDPMGBD228NLBM']

In [22]:
second_series_dicts = []
for series in second_data_series:
    data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
    series_json = data.json()
    series_json_obs=series_json['observations']
    series_dates = []
    series_values = []
    for each_item in range(len(series_json_obs)):
        series_dates.append(series_json_obs[each_item]['date'])
        series_values.append(series_json_obs[each_item]['value'])
    series_dict = dict(zip(series_dates,series_values))
    second_series_dicts.append(series_dict)

In [23]:
second_data_series_two = ['PCEPI','FPCPITOTLZGUSA','TREAST','USEPUINDXD','CUUR0000SEHA','NETEXP','PCOPPUSDM','CPIFABSL'\
                          ,'PCUOMFGOMFG']

In [24]:
second_series_two_dicts = []
for series in second_data_series_two:
    data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
    series_json = data.json()
    series_json_obs=series_json['observations']
    series_dates = []
    series_values = []
    for each_item in range(len(series_json_obs)):
        series_dates.append(series_json_obs[each_item]['date'])
        series_values.append(series_json_obs[each_item]['value'])
    series_dict = dict(zip(series_dates,series_values))
    second_series_two_dicts.append(series_dict)

In [25]:
#Dictionaries for creating/naming dataframes and columns
df_names_pull_indices_second = {0:['stock_market_cap_df','Stock_Market_Cap'],
                                1:['um_inf_expect_df', 'Inf_Expectation'],
                                2:['fed_assets_df','Fed_Assets'],
                                3:['oil_df','Price_per_Barrel'],
                                4:['core_cpi_df','CPI_Core'],
                                5:['ppi_comm_df','PPI_All_Commodities'],
                                6:['fin_stress_df','Financial_Stress'],
                                7:['urban_cpi_df','CPI_Urban'],
                                8:['medical_cpi_df','CPI_Medical'],
                                9:['trade_balance_df','Net_Trade'],
                                10:['gnp_df','GNP']}

        ######## Above items stored in df_dict.  Calc errors for oil, stock market cap and GNP#######

df_names_pull_indices_second_two = {0:['pce_index_df','PCE_Index'],
                                   1:['inf_conprice_df','Inflation_Consumer_Price'],
                                   2:['fr_held_treasuries','Fed_Res_Held_Treasuries'],
                                   3:['econ_uncer_df','Economic_Uncertainty'],
                                   4:['cpi_rent_df','CPI_Primary_Rent'],
                                   5:['netexports_df','Net_Exports'],
                                   6:['copper_df','Copper_Price'],
                                   7:['food_bev_cpi_df','CPI_Food_Bev'],
                                   8:['ppi_manufac_df','PPI_Manufacturing']}

        ######## Above items stored in df_dict_two.  Calc errors for PCE Index, Economic Uncertainty#######

In [26]:
df_dict={}
for (keys, values) in df_names_pull_indices_second.items():
    calc = second_series_dicts[keys]
    calc_df = pd.DataFrame(calc.items())
    calc_df = calc_df.rename(columns={0:'Date',1:values[1]})
    calc_column_values = calc_df[values[1]].tolist()
    calc_monthly_change = [0]
    calc_pct_change = [0]
    change_index = 1
    for m in range(len(calc_column_values)-1):
        try:
            calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
            calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                         float(calc_column_values[change_index-1])*100))
            change_index=change_index+1
        except:
            print(f'calcs on {values} didnt work')
    try:
        calc_df[f'{values[1]}_Change'] = calc_monthly_change
        calc_df[f'{values[1]}_Pct_Change'] = calc_pct_change
    except:
        print(f'couldnt add a column in {values}')
    clean_df = calc_df.set_index(['Date'])
    df_dict[values[0]]=clean_df
    
    #Change line 1 dictionary to pull indices and names, line 2 "series_dicts" for which API pull
    #calcs on oil, gnp, stock market cap NOT WORKING

calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_Market_Cap'] didnt work
calcs on ['stock_market_cap_df', 'Stock_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_Barrel'] didnt work
calcs on ['oil_df', 'Price_per_

calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['gnp_df', 'GNP'] didnt work
calcs on ['g

In [27]:
#Pull DFs from dictionaries to have standalone DFs
stock_market_cap_df=df_dict['stock_market_cap_df']
um_inf_expect_df=df_dict['um_inf_expect_df']
fed_assets_df=df_dict['fed_assets_df']
oil_df=df_dict['oil_df']
core_cpi_df=df_dict['core_cpi_df']
ppi_comm_df=df_dict['ppi_comm_df']
fin_stress_df=df_dict['fin_stress_df']
urban_cpi_df=df_dict['urban_cpi_df']
medical_cpi_df=df_dict['medical_cpi_df']
trade_balance_df=df_dict['trade_balance_df']
gnp_df=df_dict['gnp_df']

In [28]:
stock_market_cap_df=stock_market_cap_df.iloc[21:]
calcs_df=stock_market_cap_df
calcs_column_values = calcs_df['Stock_Market_Cap'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Stock_Market_Cap_Change'] = calcs_monthly_change
    calcs_df['Stock_Market_Cap_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
stock_market_cap_df=calcs_df
# stock_market_cap_df

<ipython-input-28-818d1a44f847>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Stock_Market_Cap_Change'] = calcs_monthly_change
<ipython-input-28-818d1a44f847>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Stock_Market_Cap_Pct_Change'] = calcs_pct_change


In [29]:
oil_df=oil_df.drop(index='1986-02-17')
oil_df=oil_df.loc[oil_df.loc[:,'Price_per_Barrel']!='.',:]
# oil_df

In [30]:
calcs_df=oil_df
calcs_column_values = calcs_df['Price_per_Barrel'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Price_per_Barrel_Change'] = calcs_monthly_change
    calcs_df['Price_per_Barrel_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
oil_df=calcs_df
oil_df

,Price_per_Barrel,Price_per_Barrel_Change,Price_per_Barrel_Pct_Change
Date,,,
1986-01-02,25.56,0.00,0.000000
1986-01-03,26.00,0.44,1.721440
1986-01-06,26.53,0.53,2.038462
1986-01-07,25.85,-0.68,-2.563136
1986-01-08,25.87,0.02,0.077369
...,...,...,...
2021-04-06,59.34,0.61,1.038651
2021-04-07,59.77,0.43,0.724638
2021-04-08,59.61,-0.16,-0.267693


In [31]:
gnp_df=gnp_df.loc[gnp_df.loc[:,'GNP']!='.',:]
calcs_df=gnp_df
calcs_column_values = calcs_df['GNP'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['GNP_Change'] = calcs_monthly_change
    calcs_df['GNP_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
gnp_df=calcs_df
# gnp_df

<ipython-input-31-de05aebc1741>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['GNP_Change'] = calcs_monthly_change
<ipython-input-31-de05aebc1741>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['GNP_Pct_Change'] = calcs_pct_change


In [32]:
#Mass dataframe creation and calculation with error handling
df_dict_two={}
for (keys, values) in df_names_pull_indices_second_two.items():
    calc = second_series_two_dicts[keys]
    calc_df = pd.DataFrame(calc.items())
    calc_df = calc_df.rename(columns={0:'Date',1:values[1]})
    calc_column_values = calc_df[values[1]].tolist()
    calc_monthly_change = [0]
    calc_pct_change = [0]
    change_index = 1
    for m in range(len(calc_column_values)-1):
        try:
            calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
            calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                         float(calc_column_values[change_index-1])*100))
            change_index=change_index+1
        except:
            print(f'calcs on {values} didnt work')
    try:
        calc_df[f'{values[1]}_Change'] = calc_monthly_change
        calc_df[f'{values[1]}_Pct_Change'] = calc_pct_change
    except:
        print(f'couldnt add a column in {values}')
    clean_df = calc_df.set_index(['Date'])
    df_dict_two[values[0]]=clean_df
    
    #Change line 1 dictionary to pull indices and names, line 2 "series_dicts" for which API pull
    #cals on oil, gnp, stock market cap

calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] didnt work
calcs on ['cpi_rent_df', 'CPI_Primary_Rent'] did

calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
calcs on ['copper_df', 'Copper_Price'] didnt work
couldnt add a column in ['copper_df', 'Copper_Price']
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt work
calcs on ['ppi_manufac_df', 'PPI_Manufacturing'] didnt w

In [33]:
#Pull DFs from dictionaries to have standalone DFs
pce_index_df=df_dict_two['pce_index_df']
inf_conprice_df=df_dict_two['inf_conprice_df']
fr_held_treasuries=df_dict_two['fr_held_treasuries']
econ_uncer_df=df_dict_two['econ_uncer_df']
cpi_rent_df=df_dict_two['cpi_rent_df']
netexports_df=df_dict_two['netexports_df']
copper_df=df_dict_two['copper_df']
food_bev_cpi_df=df_dict_two['food_bev_cpi_df']
ppi_manufac_df=df_dict_two['ppi_manufac_df']

In [34]:
#Clean DFs and perform calculations that failed before
cpi_rent_df=cpi_rent_df.loc[cpi_rent_df.loc[:,'CPI_Primary_Rent']!='.',:]
calcs_df=cpi_rent_df
calcs_column_values = calcs_df['CPI_Primary_Rent'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['CPI_Primary_Rent_Change'] = calcs_monthly_change
    calcs_df['CPI_Primary_Rent_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
cpi_rent_df=calcs_df
# cpi_rent_df

<ipython-input-34-76f439fded5b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['CPI_Primary_Rent_Change'] = calcs_monthly_change
<ipython-input-34-76f439fded5b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['CPI_Primary_Rent_Pct_Change'] = calcs_pct_change


In [35]:
netexports_df=netexports_df.loc[netexports_df.loc[:,'Net_Exports']!='.',:]
calcs_df=netexports_df
calcs_column_values = calcs_df['Net_Exports'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Net_Exports_Change'] = calcs_monthly_change
    calcs_df['Net_Exports_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
netexports_df=calcs_df
# netexports_df

<ipython-input-35-93fab1ed310a>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Net_Exports_Change'] = calcs_monthly_change
<ipython-input-35-93fab1ed310a>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Net_Exports_Pct_Change'] = calcs_pct_change


In [36]:
copper_df=copper_df.loc[copper_df.loc[:,'Copper_Price']!='.',:]
calcs_df=copper_df
calcs_column_values = calcs_df['Copper_Price'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Copper_Price_Change'] = calcs_monthly_change
    calcs_df['Copper_Price_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
copper_df=calcs_df
# copper_df

<ipython-input-36-e4b1f38588fd>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Copper_Price_Change'] = calcs_monthly_change
<ipython-input-36-e4b1f38588fd>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Copper_Price_Pct_Change'] = calcs_pct_change


In [37]:
ppi_manufac_df=ppi_manufac_df.loc[ppi_manufac_df.loc[:,'PPI_Manufacturing']!='.',:]
calcs_df=ppi_manufac_df
calcs_column_values = calcs_df['PPI_Manufacturing'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['PPI_Manufacturing_Change'] = calcs_monthly_change
    calcs_df['PPI_Manufacturing_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
ppi_manufac_df=calcs_df
# ppi_manufac_df

<ipython-input-37-b1a827f869bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['PPI_Manufacturing_Change'] = calcs_monthly_change
<ipython-input-37-b1a827f869bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['PPI_Manufacturing_Pct_Change'] = calcs_pct_change


In [38]:
#Lists for data pulls

third_data_series = ['UNRATE','T5YIFR','PSAVERT','MEHOINUSA672N','MSPUS','GDPDEF','WPU0911','UMCSENT','BUSLOANS','WPU101707'\
                     ,'CES0500000003','HDTGPDUSQ163N','PCU325211325211','DPSACBW027SBOG','WTREGEN','TLAACBW027SBOG','WPU101'\
                     ,'W068RCQ027SBEA','TOTRESNS','PMSAVE','IMPGSC1','TTLCONS','OPHNFB','FYGFDPUN']

fourth_data_series = ['PCE','CIVPART','T10YIE','PCEC96','GPDIC1','CP','PCEDG','GPDI','T30YIEM','JTSJOL','TOTBKCR'\
                      ,'WSHOMCB','PCEND','PCESV','RRVRUSQ156N','DRSFRMACBS','DSPIC96','TDSP','RHORUSQ156N','DRCCLACBS','ECOMPCTSA'\
                      ,'MORTGAGE30US','DRCLACBS','HOUST']

In [39]:
##Dictionaries for creating/naming dataframes and columns

df_names_pull_indices_third = {0:['unemployment_df','Unemployment'],
                                1:['inf_exp5yr_df', 'Inf_Expectation_5yr'],
                                2:['pers_savings_df','Personal_Savings'],
                                3:['rmed_hous_income_df','Real_Median_House_Income'],
                                4:['med_house_sales_df','Median_House_Sale_Price'],
                                5:['dgp_deflator_df','GDP_Deflator'],
                                6:['ppi_woodpulp_df','PPI_Wood_Pulp'],
                                7:['um_consumer_sent_df','Consumer_Sentiment'],
                                8:['biz_loans_df','Commercial_Industrial_Loans'],
                                9:['ppi_metals_df','PPI_Metals'],
                                10:['avg_hourly_earnings_df','Average_Hourly_Wage'],
                                11:['house_debttoGDP_df', 'House_Debt_GDP_Ratio'],
                                12:['ppi_plastics_df','PPI_Plastics_Resins'],
                                13:['comm_bank_deposits_df','Commercial_Bank_Deposits'],
                                14:['nonreserve_fed_liabilities_df','Fed_Liabilities(non_reserve-deposits)'],
                                15:['comm_bank_assets_df','Commercial_Bank_Assets'],
                                16:['ppi_iron_steel_df','PPI_Iron_Steel'],
                                17:['gov_expenditures_df','Government_Expenditures'],
                                18:['reserve_balances_df','Reserve_Balances'],
                                19:['personal_savings_df','Personal_Savings'],
                                20:['real_imports_df','Real_Imports'],
                                21:['construction_spending_df', 'Construction_Spending'],
                                22:['real_hourly_output_df','Real_Output_Hour'],
                                23:['debt_heldby_public_df','Federal_Debt_Held_by_Public']}

df_names_pull_indices_fourth = {0:['pce_df','PCE'],
                                1:['labor_particip_df', 'Labor_Participation_Rate'],
                                2:['tenyr_inf_breakeven_df','Ten_Year_Breakeven_Inflation'],
                                3:['real_PCE_df','Real_PCE'],
                                4:['real_grosdompriv_inv_df','Real_Gross_Domestic_Private_Investment'],
                                5:['corporate_profits_df','Corporate_Profits_After_Tax'],
                                6:['pce_durable_df','PCE_Durable_Goods'],
                                7:['grosdompriv_inv_df','Gross_Domestic_Private_Investment'],
                                8:['inf_30yr_breakeven_df','_30_Year_Breakeven_Inflation'],
                                9:['job_openings_df','Job_Openings_nonfarm'],
                                10:['com_bank_credit_df','Commercial_Bank_Credit'],
                                11:['fedassets_MBS_df', 'Fed_MBS'],
                                12:['pce_nondurable_df','PCE_NonDurable_Goods'],
                                13:['pce_services_df','PCE_Services'],
                                14:['rental_vacancy_rate_df','Rental_Vacancy_Rate'],
                                15:['mortgage_delinquency_df','Mortgage_Delinquency'],
                                16:['real_disp_income_df','Real_Disposable_Personal_Income'],
                                17:['house_debt_service_pmt_df','Household_Debt_Service_PmtPctGDP'],
                                18:['homeownership_rate_df','Homeownership_Rate'],
                                19:['cc_delinquency_rate_df','CreditCard_Delinquency_Rate'],
                                20:['ecommerce_df','Ecommerce_Pct_of_Totalsales'],
                                21:['mortgage_30yr_rate_df', 'ThirtyYr_Fixed_Rate_Mortgage'],
                                22:['consumer_loan_delinquency_df','Consumer_Loan_Delinquency_Rate'],
                                23:['housing_starts_df','Housing_Starts']}

In [40]:
#Mass data pull with error handling
error_list = []
third_series_dicts = []
for series in third_data_series:
    try:
        data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
        series_json = data.json()
        series_json_obs=series_json['observations']
        series_dates = []
        series_values = []
        for each_item in range(len(series_json_obs)):
            series_dates.append(series_json_obs[each_item]['date'])
            series_values.append(series_json_obs[each_item]['value'])
        series_dict = dict(zip(series_dates,series_values))
        third_series_dicts.append(series_dict)
    except:
        print(f'something went wrong pulling {series}')
        error_list.append(series)

In [41]:
df_dict_three={}
for (keys, values) in df_names_pull_indices_third.items():
    calc = third_series_dicts[keys]
    calc_df = pd.DataFrame(calc.items())
    calc_df = calc_df.rename(columns={0:'Date',1:values[1]})
    calc_column_values = calc_df[values[1]].tolist()
    calc_monthly_change = [0]
    calc_pct_change = [0]
    change_index = 1
    for m in range(len(calc_column_values)-1):
        try:
            calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
            calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                         float(calc_column_values[change_index-1])*100))
            change_index=change_index+1
        except:
            print(f'calcs on {values} didnt work')
    try:
        calc_df[f'{values[1]}_Change'] = calc_monthly_change
        calc_df[f'{values[1]}_Pct_Change'] = calc_pct_change
    except:
        print(f'couldnt add a column in {values}')
    clean_df = calc_df.set_index(['Date'])
    df_dict_three[values[0]]=clean_df

calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df

calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df

calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df', 'Inf_Expectation_5yr'] didnt work
calcs on ['inf_exp5yr_df

calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on ['um_consumer_sent_df', 'Consumer_Sentiment'] didnt work
calcs on [

In [42]:
##Pull DFs from dictionaries to have standalone DFs
unemployment_df=df_dict_three['unemployment_df']
inf_exp5yr_df=df_dict_three['inf_exp5yr_df']
pers_savings_df=df_dict_three['pers_savings_df']
rmed_hous_income_df=df_dict_three['rmed_hous_income_df']
med_house_sales_df=df_dict_three['med_house_sales_df']
dgp_deflator_df=df_dict_three['dgp_deflator_df']
ppi_woodpulp_df=df_dict_three['ppi_woodpulp_df']
um_consumer_sent_df=df_dict_three['um_consumer_sent_df']
biz_loans_df=df_dict_three['biz_loans_df']
ppi_metals_df=df_dict_three['ppi_metals_df']
avg_hourly_earnings_df=df_dict_three['avg_hourly_earnings_df']
house_debttoGDP_df=df_dict_three['house_debttoGDP_df']
ppi_plastics_df=df_dict_three['ppi_plastics_df']
comm_bank_deposits_df=df_dict_three['comm_bank_deposits_df']
nonreserve_fed_liabilities_df=df_dict_three['nonreserve_fed_liabilities_df']
comm_bank_assets_df=df_dict_three['comm_bank_assets_df']
ppi_iron_steel_df=df_dict_three['ppi_iron_steel_df']
gov_expenditures_df=df_dict_three['gov_expenditures_df']
reserve_balances_df=df_dict_three['reserve_balances_df']
personal_savings_df=df_dict_three['personal_savings_df']
real_imports_df=df_dict_three['real_imports_df']
construction_spending_df=df_dict_three['construction_spending_df']
real_hourly_output_df=df_dict_three['real_hourly_output_df']
debt_heldby_public_df=df_dict_three['debt_heldby_public_df']

In [43]:
inf_exp5yr_df=inf_exp5yr_df.loc[inf_exp5yr_df.loc[:,'Inf_Expectation_5yr']!='.',:]
calcs_df=inf_exp5yr_df
calcs_column_values = calcs_df['Inf_Expectation_5yr'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Inf_Expectation_5yr_Change'] = calcs_monthly_change
    calcs_df['Inf_Expectation_5yr_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
inf_exp5yr_df=calcs_df
# inf_exp5yr_df

<ipython-input-43-5118938a9790>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Inf_Expectation_5yr_Change'] = calcs_monthly_change
<ipython-input-43-5118938a9790>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Inf_Expectation_5yr_Pct_Change'] = calcs_pct_change


In [44]:
um_consumer_sent_df=um_consumer_sent_df.loc[um_consumer_sent_df.loc[:,'Consumer_Sentiment']!='.',:]
calcs_df=um_consumer_sent_df
calcs_column_values = calcs_df['Consumer_Sentiment'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Consumer_Sentiment_Change'] = calcs_monthly_change
    calcs_df['Consumer_Sentiment_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
um_consumer_sent_df=calcs_df
# um_consumer_sent_df

<ipython-input-44-24090c57a320>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Consumer_Sentiment_Change'] = calcs_monthly_change
<ipython-input-44-24090c57a320>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Consumer_Sentiment_Pct_Change'] = calcs_pct_change


In [45]:
#Mass data pull with error handling
#error_list = []
fourth_series_dicts = []
for series in fourth_data_series:
    try:
        data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
        series_json = data.json()
        series_json_obs=series_json['observations']
        series_dates = []
        series_values = []
        for each_item in range(len(series_json_obs)):
            series_dates.append(series_json_obs[each_item]['date'])
            series_values.append(series_json_obs[each_item]['value'])
        series_dict = dict(zip(series_dates,series_values))
        fourth_series_dicts.append(series_dict)
    except:
        print(f'something went wrong pulling {series}')
        error_list.append(series)

In [46]:
df_dict_four={}
for (keys, values) in df_names_pull_indices_fourth.items():
    calc = fourth_series_dicts[keys]
    calc_df = pd.DataFrame(calc.items())
    calc_df = calc_df.rename(columns={0:'Date',1:values[1]})
    calc_column_values = calc_df[values[1]].tolist()
    calc_monthly_change = [0]
    calc_pct_change = [0]
    change_index = 1
    for m in range(len(calc_column_values)-1):
        try:
            calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
            calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                         float(calc_column_values[change_index-1])*100))
            change_index=change_index+1
        except:
            print(f'calcs on {values} didnt work')
    try:
        calc_df[f'{values[1]}_Change'] = calc_monthly_change
        calc_df[f'{values[1]}_Pct_Change'] = calc_pct_change
    except:
        print(f'couldnt add a column in {values}')
    clean_df = calc_df.set_index(['Date'])
    df_dict_four[values[0]]=clean_df

calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation

calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation

calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation

calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation'] didnt work
calcs on ['tenyr_inf_breakeven_df', '10_Year_Breakeven_Inflation

calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_Domestic_Private_Investment'] didnt work
calcs on ['grosdompriv_inv_df', 'Gross_D

calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['fedassets_MBS_df', 'Fed_MBS'] didnt work
calcs on ['f

In [47]:
#Pull DFs from dictionaries to have standalone DFs
pce_df=df_dict_four['pce_df']
labor_particip_df=df_dict_four['labor_particip_df']
tenyr_inf_breakeven_df=df_dict_four['tenyr_inf_breakeven_df']
real_PCE_df=df_dict_four['real_PCE_df']
real_grosdompriv_inv_df=df_dict_four['real_grosdompriv_inv_df']
corporate_profits_df=df_dict_four['corporate_profits_df']
pce_durable_df=df_dict_four['pce_durable_df']
grosdompriv_inv_df=df_dict_four['grosdompriv_inv_df']
inf_30yr_breakeven_df=df_dict_four['inf_30yr_breakeven_df']
job_openings_df=df_dict_four['job_openings_df']
com_bank_credit_df=df_dict_four['com_bank_credit_df']
fedassets_MBS_df=df_dict_four['fedassets_MBS_df']
pce_nondurable_df=df_dict_four['pce_nondurable_df']
pce_services_df=df_dict_four['pce_services_df']
rental_vacancy_rate_df=df_dict_four['rental_vacancy_rate_df']
mortgage_delinquency_df=df_dict_four['mortgage_delinquency_df']
real_disp_income_df=df_dict_four['real_disp_income_df']
house_debt_service_pmt_df=df_dict_four['house_debt_service_pmt_df']
homeownership_rate_df=df_dict_four['homeownership_rate_df']
cc_delinquency_rate_df=df_dict_four['cc_delinquency_rate_df']
ecommerce_df=df_dict_four['ecommerce_df']
mortgage_30yr_rate_df=df_dict_four['mortgage_30yr_rate_df']
consumer_loan_delinquency_df=df_dict_four['consumer_loan_delinquency_df']
housing_starts_df=df_dict_four['housing_starts_df']

In [48]:
tenyr_inf_breakeven_df=tenyr_inf_breakeven_df.loc[tenyr_inf_breakeven_df.loc[:,'10_Year_Breakeven_Inflation']!='.',:]
calcs_df=tenyr_inf_breakeven_df
calcs_column_values = calcs_df['10_Year_Breakeven_Inflation'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['10_Year_Breakeven_Inflation_Change'] = calcs_monthly_change
    calcs_df['10_Year_Breakeven_Inflation_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
tenyr_inf_breakeven_df=calcs_df
# tenyr_inf_breakeven_df

<ipython-input-48-3b60769801a1>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['10_Year_Breakeven_Inflation_Change'] = calcs_monthly_change
<ipython-input-48-3b60769801a1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['10_Year_Breakeven_Inflation_Pct_Change'] = calcs_pct_change


In [49]:
real_PCE_df=real_PCE_df.loc[real_PCE_df.loc[:,'Real_PCE']!='.',:]
calcs_df=real_PCE_df
calcs_column_values = calcs_df['Real_PCE'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Real_PCE_Change'] = calcs_monthly_change
    calcs_df['Real_PCE_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
real_PCE_df=calcs_df
# real_PCE_df

<ipython-input-49-f7e3b595e06c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Real_PCE_Change'] = calcs_monthly_change
<ipython-input-49-f7e3b595e06c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Real_PCE_Pct_Change'] = calcs_pct_change


In [50]:
corporate_profits_df=corporate_profits_df.loc[corporate_profits_df.loc[:,'Corporate_Profits_After_Tax']!='.',:]
calcs_df=corporate_profits_df
calcs_column_values = calcs_df['Corporate_Profits_After_Tax'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Corporate_Profits_After_Tax_Change'] = calcs_monthly_change
    calcs_df['Corporate_Profits_After_Tax_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
corporate_profits_df=calcs_df
# corporate_profits_df

<ipython-input-50-b2555122b5b6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Corporate_Profits_After_Tax_Change'] = calcs_monthly_change
<ipython-input-50-b2555122b5b6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Corporate_Profits_After_Tax_Pct_Change'] = calcs_pct_change


In [51]:
grosdompriv_inv_df=grosdompriv_inv_df.loc[grosdompriv_inv_df.loc[:,'Gross_Domestic_Private_Investment']!='.',:]
calcs_df=grosdompriv_inv_df
calcs_column_values = calcs_df['Gross_Domestic_Private_Investment'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Gross_Domestic_Private_Investment_Change'] = calcs_monthly_change
    calcs_df['Gross_Domestic_Private_Investment_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
grosdompriv_inv_df=calcs_df
# grosdompriv_inv_df

<ipython-input-51-9892fc0efc12>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Gross_Domestic_Private_Investment_Change'] = calcs_monthly_change
<ipython-input-51-9892fc0efc12>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Gross_Domestic_Private_Investment_Pct_Change'] = calcs_pct_change


In [53]:
# fedassets_MBS_df=fedassets_MBS_df.loc[fedassets_MBS_df.loc[:,'Fed_MBS']!='.',:]
calcs_df=fedassets_MBS_df
calcs_column_values = calcs_df['Fed_MBS'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
#         calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
#                                      float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        calcs_monthly_change.append("0")
#         calcs_pct_change.append("0")
try:
    calcs_df['Fed_MBS_Change'] = calcs_monthly_change
#     calcs_df['Fed_MBS_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
fedassets_MBS_df=calcs_df
# fedassets_MBS_df=fedassets_MBS_df.drop(columns='Fed_MBS_Pct_Change')
# fedassets_MBS_df

In [54]:
pce_services_df=pce_services_df.loc[pce_services_df.loc[:,'PCE_Services']!='.',:]
calcs_df=pce_services_df
calcs_column_values = calcs_df['PCE_Services'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['PCE_Services_Change'] = calcs_monthly_change
    calcs_df['PCE_Services_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
pce_services_df=calcs_df
# pce_services_df

<ipython-input-54-c67153854f63>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['PCE_Services_Change'] = calcs_monthly_change
<ipython-input-54-c67153854f63>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['PCE_Services_Pct_Change'] = calcs_pct_change


In [55]:
# fifth_data_series =['GOLDPMGBD228NLBM']
# fifth_series_dicts = []
# for series in fifth_data_series:
#     data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
#     series_json = data.json()
#     series_json_obs=series_json['observations']
#     series_dates = []
#     series_values = []
#     for each_item in range(len(series_json_obs)):
#         series_dates.append(series_json_obs[each_item]['date'])
#         series_values.append(series_json_obs[each_item]['value'])
#     series_dict = dict(zip(series_dates,series_values))
#     second_series_dicts.append(series_dict)

In [124]:
fifth_data_series = ['WPU0811','WFRBST01134','PWHEAMTUSDQ','GPSAVE','CPIAPPSL','QUSR628BIS','CPIHOSNS','CUUR0000SETA01','IMPGS'\
                     ,'M318501Q027NBEA','WFRBLB50107','PCEDGC96','CONSUMER','NCBCMDPMVCE','WPU10170502','PMAIZMTUSDM'\
                     ,'WFRBLT01026','B087RC1Q027SBEA','INDCPIALLMINMEI','PIORECRUSDM','CPITRNSL','PCU484121484121','PALUMUSDM'\
                     ,'PRUBBUSDM','CASACBW027SBOG','RESPPANWW','WPU081','PCU32733273','NONM1','IMPCH','PCU44414441','IR'\
                     ,'B075RC1Q027SBEA','PCU33443344','APU0000708111','A019RE1A156NBEA']

In [125]:
df_names_pull_indices_fifth = {0:['ppi_wood_lumber_df','PPI_Wood_Lumber'],
                                1:['wealth_share_1pct', 'Wealth_Share_Top1pct'],
                                2:['global_wheat_price_df','Global_Wheat_Price'],
                                3:['gross_prvt_saving_df','Gross_Private_Saving'],
                                4:['cpi_aparrel_df','CPI_Apparel_Cities'],
                                5:['real_residential_property_prices_df','Real_Residential_Property_Price'],
                                6:['cpi_city_housing_df','CPI_Housing_Cities'],
                                7:['cpi_vehicles_df','CPI_Vehicles'],
                                8:['imports_goods_services_df','Imports_Goods&Services'],
                                9:['surplus_deficit_df','Federal_Surplus_Deficit'],
                                10:['bottom50_networth_df','Bottom_50pct_Net_Worth'],
                                11:['real_pce_durable_df','Real_PCE_Durable_Goods'],
                                12:['consumer_loans_df','Consumer_Loans_Com_Banks'],
                                13:['corporate_equ_debt_pct_df','Debt_as_Pct_Corporate_Equities'],
                                14:['ppi_steelwire_df','PPI_Steel_Wire'],
                                15:['global_corn_price_df','Global_Corn_Price'],
                                16:['wealth_total_1pct_df','Wealth_Total_Top1pct'],
                                17:['gov_transfer_pmt_df','Government_Transfer_Payments'],
                                18:['cpi_india_df','CPI_India'],
                                19:['global_iron_price_df','Global_Iron_Price'],
                                20:['cpi_urban_transportation_df', 'CPI_Urban_Transportation'],
                                21:['ppi_freight_df','PPI_Freight'],
                                22:['global_aluminum_price_df','Global_Aluminum_Price'],
                                23:['global_rubber_price_df','Global_Rubber_Price'],
                                24:['cash_assets_combanks_df','Commercial_Bank_Cash_Assets'],
                                25:['total_fed_assets_df','Total_Fed_Assets'],
                                26:['ppi_lumber_df','PPI_Lumber'],
                                27:['ppi_cementconcrete_df','PPI_Cement_Concrete'],
                                28:['nonM1_m2_df','Non_M1_Components_M2'],
                                29:['imports_china_df','Imports_from_China'],
                                30:['ppi_building_materials_df', 'PPI_Building_Materials'],
                                31:['import_price_index_commod_df','All_Commodities_Import_Price_Index'],
                                32:['corporate_incometax_receipts_df','Corporate_Income_Tax_Receipts'],
                                33:['ppi_semiconductors_df','PPI_Semiconductors_Electronics'],
                                34:['cpi_eggs_df','CPI_Eggs'],
                                35:['net_exports_pctGDP_df','Net_Exports_PctofGDP'],
                                }

In [126]:
fifth_series_dicts = []
for series in fifth_data_series:
    try:
        data = requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
        series_json = data.json()
        series_json_obs=series_json['observations']
        series_dates = []
        series_values = []
        for each_item in range(len(series_json_obs)):
            series_dates.append(series_json_obs[each_item]['date'])
            series_values.append(series_json_obs[each_item]['value'])
        series_dict = dict(zip(series_dates,series_values))
        fifth_series_dicts.append(series_dict)
    except:
        print(f'something went wrong pulling {series}')
        error_list.append(series)

In [127]:
df_dict_five={}
for (keys, values) in df_names_pull_indices_fifth.items():
    calc = fifth_series_dicts[keys]
    calc_df = pd.DataFrame(calc.items())
    calc_df = calc_df.rename(columns={0:'Date',1:values[1]})
    calc_column_values = calc_df[values[1]].tolist()
    calc_monthly_change = [0]
    calc_pct_change = [0]
    change_index = 1
    for m in range(len(calc_column_values)-1):
        try:
            calc_monthly_change.append(float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))
            calc_pct_change.append(((float(calc_column_values[change_index])-float(calc_column_values[change_index-1]))/
                                         float(calc_column_values[change_index-1])*100))
            change_index=change_index+1
        except:
            print(f'calcs on {values} didnt work')
    try:
        calc_df[f'{values[1]}_Change'] = calc_monthly_change
        calc_df[f'{values[1]}_Pct_Change'] = calc_pct_change
    except:
        print(f'couldnt add a column in {values}')
    clean_df = calc_df.set_index(['Date'])
    df_dict_five[values[0]]=clean_df

calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat_Price'] didnt work
calcs on ['global_wheat_price_df', 'Global_Wheat

calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_durable_df', 'Real_PCE_Durable_Goods'] didnt work
calcs on ['real_pce_

calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work
calcs on ['cpi_india_df', 'CPI_India'] didnt work


calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 'Global_Aluminum_Price'] didnt work
calcs on ['global_aluminum_price_df', 

calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work
calcs on ['import_price_index_commod_df', 'All_Commodities_Import_Price_Index'] didnt work

In [134]:
# ppi_wood_lumber_df=df_dict_five["ppi_wood_lumber_df"]
# wealth_share_1pct=df_dict_five["wealth_share_1pct"]
# global_wheat_price_df=df_dict_five["global_wheat_price_df"]
# gross_prvt_saving_df=df_dict_five["gross_prvt_saving_df"]
# cpi_aparrel_df=df_dict_five["cpi_aparrel_df"]
# real_residential_property_prices_df=df_dict_five["real_residential_property_prices_df"]
# cpi_city_housing_df=df_dict_five["cpi_city_housing_df"]
# cpi_vehicles_df=df_dict_five["cpi_vehicles_df"]
# imports_goods_services_df=df_dict_five["imports_goods_services_df"]
# surplus_deficit_df=df_dict_five["surplus_deficit_df"]
# bottom50_networth_df=df_dict_five["bottom50_networth_df"]
# real_pce_durable_df=df_dict_five["real_pce_durable_df"]
# consumer_loans_df=df_dict_five["consumer_loans_df"]
# corporate_equ_debt_pct_df=df_dict_five["corporate_equ_debt_pct_df"]
# ppi_steelwire_df=df_dict_five["ppi_steelwire_df"]
# global_corn_price_df=df_dict_five["global_corn_price_df"]
# wealth_total_1pct_df=df_dict_five["wealth_total_1pct_df"]
# gov_transfer_pmt_df=df_dict_five["gov_transfer_pmt_df"]
# cpi_india_df=df_dict_five["cpi_india_df"]
# global_iron_price_df=df_dict_five["global_iron_price_df"]
# cpi_urban_transportation_df=df_dict_five["cpi_urban_transportation_df"]
# ppi_freight_df=df_dict_five["ppi_freight_df"]
# global_aluminum_price_df=df_dict_five["global_aluminum_price_df"]
# global_rubber_price_df=df_dict_five["global_rubber_price_df"]
# cash_assets_combanks_df=df_dict_five["cash_assets_combanks_df"]
# total_fed_assets_df=df_dict_five["total_fed_assets_df"]
# ppi_lumber_df=df_dict_five["ppi_lumber_df"]
# ppi_cementconcrete_df=df_dict_five["ppi_cementconcrete_df"]
# nonM1_m2_df=df_dict_five["nonM1_m2_df"]
# imports_china_df=df_dict_five["imports_china_df"]
# ppi_building_materials_df=df_dict_five["ppi_building_materials_df"]
# import_price_index_commod_df=df_dict_five["import_price_index_commod_df"]
# corporate_incometax_receipts_df=df_dict_five["corporate_incometax_receipts_df"]
# ppi_semiconductors_df=df_dict_five["ppi_semiconductors_df"]
# cpi_eggs_df=df_dict_five["cpi_eggs_df"]
# net_exports_pctGDP_df=df_dict_five["net_exports_pctGDP_df"]

In [ ]:
#List of calc error fixes for df_dict_five to run in cell above
# global_wheat_price_df
# cpi_vehicles_df
# imports_goods_services_df
# real_pce_durable_df
# corporate_equ_debt_pct_df
# global_corn_price_df
# cpi_india_df
# global_iron_price_df
# global_aluminum_price_df
# global_rubber_price_df
# imports_china_df
# import_price_index_commod_df
# net_exports_pctGDP_df

In [186]:
###### Calcs on df_dict_five df's with calc errors due to decimals in cells
net_exports_pctGDP_df=net_exports_pctGDP_df.loc[net_exports_pctGDP_df.loc[:,'Net_Exports_PctofGDP']!='.',:]
calcs_df=net_exports_pctGDP_df
calcs_column_values = calcs_df['Net_Exports_PctofGDP'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Net_Exports_PctofGDP_Change'] = calcs_monthly_change
    calcs_df['Net_Exports_PctofGDP_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
net_exports_pctGDP_df=calcs_df

calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt work
calcs didnt wo

- Sat: Finalize DB, Finalize Index, Plan Dashboard structure
- Sun & Mon: Build basic dashboard (not connected to db; test with one sheet)
- Tues: Build Flask, Connect to DB
- Wed: Add style
- Thu: Analysis
- Friday: Analysis

In [ ]:
# Query for DJIA and NDAQ daily stock data
# tickers = ['DJI','NDAQ','SWTX']
# stock_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=SWTX&datatype=json&apikey={alvan_key}'
# stock_pull = requests.get(stock_url)
# dji_json = stock_pull.json()
# dji_json

In [190]:
# from api_keys import quandl_key, alvan_key

In [223]:
# metals_list=['XAU','XAG']
# stocks_list=['NDAQ','DJIA']
# gold_silver=['GOLDPMGBD228NLBM','SLVPRUSD']

In [224]:
# sixth_series_dicts = []
# for series in gold_silver:
#     try:
# #         data = requests.get(f'https://www.quandl.com/api/v3/datasets/WIKI/{series}/data.json?api_key={quandl_key}')
#         data= requests.get(f'https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key={fred_key}&file_type=json')
#         series_json = data.json()
# #         series_json_obs=series_json['observations']
# #         series_dates = []
# #         series_values = []
# #         for each_item in range(len(series_json_obs)):
# #             series_dates.append(series_json_obs[each_item]['date'])
# #             series_values.append(series_json_obs[each_item]['value'])
# #         series_dict = dict(zip(series_dates,series_values))
# #         fifth_series_dicts.append(series_dict)
#     except:
#         print(f'something went wrong pulling {series}')
# #         error_list.append(series)

In [225]:
series_json

{'error_code': 404,
 'error_message': 'Not Found.  The data for this series is not available via the API;\n                 the data can be found on the FRED website.'}

In [246]:
### Had to manually download stock/gold csvs from WSJ and Gold from FRED (not available through API)
djia_path = os.path.join("DJIA.csv")
nasdaq_path = os.path.join("Nasdaq.csv")
gold_path=os.path.join("Gold.csv")
sp500_path=os.path.join("S&P500.csv")
djia_data = pd.read_csv(djia_path)
nasdaq_data = pd.read_csv(nasdaq_path)
gold_data=pd.read_csv(gold_path)
sp500_data=pd.read_csv(sp500_path)

In [258]:
djia_df=djia_data.set_index('Date')
nasdaq_df=nasdaq_data.set_index('Date')
gold_df=gold_data.set_index('Date')
sp500_df=sp500_data.set_index('Date')

In [259]:
djia_df=djia_df[::-1]
nasdaq_df=nasdaq_df[::-1]
sp500_df=sp500_df[::-1]

In [296]:
gold_df=gold_df.loc[gold_df.loc[:,'Gold_Price']!='.',:]
calcs_df=gold_df
calcs_column_values = calcs_df['Gold_Price'].tolist()
calcs_monthly_change = [0]
calcs_pct_change = [0]
change_index = 1
for m in range(len(calcs_column_values)-1):
    try:
        calcs_monthly_change.append(float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))
        calcs_pct_change.append(((float(calcs_column_values[change_index])-float(calcs_column_values[change_index-1]))/
                                     float(calcs_column_values[change_index-1])*100))
        change_index=change_index+1
    except:
        print(f'calcs didnt work')
try:
    calcs_df['Gold_Price_Change'] = calcs_monthly_change
    calcs_df['Gold_Price_Pct_Change'] = calcs_pct_change
except:
    print(f'couldnt add a column')
gold_df=calcs_df

<ipython-input-296-2ba5a2a35363>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Gold_Price_Change'] = calcs_monthly_change
<ipython-input-296-2ba5a2a35363>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calcs_df['Gold_Price_Pct_Change'] = calcs_pct_change


In [297]:
gold_df

,Gold_Price,Gold_Price_Change,Gold_Price_Pct_Change
Date,,,
4/1/1968,37.7,0.00,0.000000
4/2/1968,37.3,-0.40,-1.061008
4/3/1968,37.6,0.30,0.804290
4/4/1968,36.95,-0.65,-1.728723
4/5/1968,37,0.05,0.135318
...,...,...,...
4/12/2021,1732.85,-8.35,-0.479554
4/13/2021,1747.95,15.10,0.871397
4/14/2021,1735.55,-12.40,-0.709402


In [262]:
m1m2_list=[m2_df,nonM1_m2_df]
m1m2_df=m1_df
for each_df in m1m2_list:
    m1m2_df = pd.merge(m1m2_df,each_df, how="outer", on=['Date'])
m1m2_df.to_csv('DFs_for_DB/m1m2.csv')

In [263]:
velocity_df = pd.merge(m1v_df, m2v_df, how="outer", on=['Date'])
velocity_df.to_csv('DFs_for_DB/velocity.csv')

In [389]:
consumer_monthly = [pce_index_df,unemployment_df,pers_savings_df,avg_hourly_earnings_df,personal_savings_df,um_consumer_sent_df\
                    ,cpi_vehicles_df,cpi_eggs_df,cpi_aparrel_df,cpi_city_housing_df,real_pce_durable_df,cpi_urban_transportation_df\
                    ,pce_df,labor_particip_df,real_PCE_df,pce_durable_df,job_openings_df,pce_nondurable_df,real_disp_income_df]
consumer_monthly_df=pop_df
for each_df in consumer_monthly:
    consumer_monthly_df=pd.merge(consumer_monthly_df,each_df, how='outer',on=['Date'])
consumer_monthly_df.to_csv('DFs_for_DB/consumer_monthly.csv')

In [285]:
ppi_monthly = [ppi_comm_df,ppi_woodpulp_df,ppi_metals_df,copper_df,ppi_plastics_df,ppi_iron_steel_df,global_wheat_price_df,ppi_steelwire_df\
              ,ppi_freight_df,global_aluminum_price_df,ppi_building_materials_df,ppi_wood_lumber_df,global_corn_price_df,global_iron_price_df\
               ,global_rubber_price_df,ppi_lumber_df,ppi_cementconcrete_df,ppi_semiconductors_df]
ppi_monthly_df=ppi_manufac_df
for each_df in ppi_monthly:
    ppi_monthly_df=pd.merge(ppi_monthly_df,each_df,how='outer',on=['Date'])
ppi_monthly_df.to_csv('DFs_for_DB/ppi_monthly.csv')

In [266]:
gdp_quarterly = [ngdpcap_df,real_gdp_df,rgdpcap_df,dgp_deflator_df,gnp_df]
gdp_quarterly_df=gdp_df
for each_df in gdp_quarterly:
    gdp_quarterly_df=pd.merge(gdp_quarterly_df,each_df,how='outer',on=['Date'])
gdp_quarterly_df.to_csv('DFs_for_DB/gdp_quarterly.csv')
#I spelled gdp_deflator with dgp way above

In [381]:
consumers_quarterly = [med_house_sales_df,house_debttoGDP_df,real_hourly_output_df,corporate_profits_df,pce_services_df,rental_vacancy_rate_df\
                       ,mortgage_delinquency_df,house_debt_service_pmt_df,homeownership_rate_df,cc_delinquency_rate_df,wealth_share_1pct_df\
                       ,gross_prvt_saving_df,real_residential_property_prices_df,bottom50_networth_df,corporate_equ_debt_pct_df,wealth_total_1pct_df\
                       ,consumer_loan_delinquency_df]
consumers_quarterly_df = ecommerce_df
for each_df in consumers_quarterly:
    consumers_quarterly_df=pd.merge(consumers_quarterly_df,each_df,how='outer',on=['Date'])
consumers_quarterly_df.to_csv('DFs_for_DB/consumers_quarterly.csv')

In [286]:
cpi_monthly = [um_inf_expect_df,core_cpi_df,medical_cpi_df,urban_cpi_df,cpi_rent_df,food_bev_cpi_df]
cpi_monthly_df = cpi_df
for each_df in cpi_monthly:
    cpi_monthly_df=pd.merge(cpi_monthly_df,each_df,how='outer',on=['Date'])
cpi_monthly_df.to_csv('DFs_for_DB/cpi_monthly.csv')

In [269]:
banks_week_month = [fedrate_df,comm_bank_deposits_df,comm_bank_assets_df,reserve_balances_df,com_bank_credit_df,mortgage_30yr_rate_df\
                   ,consumer_loans_df,cash_assets_combanks_df]
banks_week_month_df = biz_loans_df 
for each_df in banks_week_month:
    banks_week_month_df=pd.merge(banks_week_month_df,each_df,how='outer',on=['Date'])
banks_week_month_df.to_csv('DFs_for_DB/banks_week_month.csv')

In [270]:
gov_quarterly = [debtpct_df,gov_expenditures_df,debt_heldby_public_df,frdebt_df,gov_transfer_pmt_df,surplus_deficit_df\
                 ,corporate_incometax_receipts_df]
gov_quarterly_df = debt_df
for each_df in gov_quarterly:
    gov_quarterly_df=pd.merge(gov_quarterly_df,each_df,how='outer',on=['Date'])
gov_quarterly_df.to_csv('DFs_for_DB/government_quarterly.csv')

In [271]:
fed_res_weekly = [fed_assets_df,fr_held_treasuries,nonreserve_fed_liabilities_df,fedassets_MBS_df,total_fed_assets_df]
fed_res_weekly_df = fin_stress_df
for each_df in fed_res_weekly:
    fed_res_weekly_df=pd.merge(fed_res_weekly_df,each_df,how='outer',on=['Date'])
fed_res_weekly_df.to_csv('DFs_for_DB/federal_reserve_weekly.csv')

In [272]:
foreign_trade = [netexports_df,net_exports_pctGDP_df,real_imports_df,imports_goods_services_df,cpi_india_df,imports_china_df,import_price_index_commod_df]
foreign_trade_df = trade_balance_df
for each_df in foreign_trade:
    foreign_trade_df=pd.merge(foreign_trade_df,each_df,how='outer',on=['Date'])
foreign_trade_df.to_csv('DFs_for_DB/foreign_trade_month_quarter.csv')

In [273]:
investment_month_week = [housing_starts_df,real_grosdompriv_inv_df,grosdompriv_inv_df]
investment_df = construction_spending_df
for each_df in investment_month_week:
    investment_df=pd.merge(investment_df,each_df,how='outer',on=['Date'])
investment_df.to_csv('DFs_for_DB/investment_month_quarter.csv')

In [396]:
misc_daily = [oil_df,econ_uncer_df,tenyr_inf_breakeven_df]
misc_daily_df = inf_exp5yr_df
for each_df in misc_daily:
    misc_daily_df=pd.merge(misc_daily_df,each_df,how='outer',on=['Date'])
misc_daily_df.to_csv('DFs_for_DB/misc_daily.csv')

In [391]:
misc_annual = [stock_market_cap_df,rmed_hous_income_df,inf_conprice_df]
misc_annual_df = deficit_df
for each_df in misc_annual:
    misc_annual_df=pd.merge(misc_annual_df,each_df,how='outer',on=['Date'])
misc_annual_df.to_csv('DFs_for_DB/misc_annual.csv')

In [327]:
stocks_gold_daily = [nasdaq_df,sp500_df,gold_df]
stocks_gold_daily_df = djia_df
for each_df in stocks_gold_daily:
    stocks_gold_daily_df=pd.merge(stocks_gold_daily_df,each_df,how='outer',on=['Date'])
stocks_gold_daily_df.to_csv('DFs_for_DB/stocks_gold_daily.csv')

- Import More Data Series --> run calculations
- Join Quarterly
    m1v_df, m2v_df, gdp_df, real_gdp_df, debt_df, frdebt_df, debtpct_df, rgdpcap_df, ngdpcap_df
- Join Monthly
    pop_df, fedrate_df, cpi_df
- Export big joins to csvs
- Create ERD
- Load ERD code into Postgres
- Import CSVs into Postgres

In [ ]:
[pce_index_df,unemployment_df,pers_savings_df,avg_hourly_earnings_df,personal_savings_df,um_consumer_sent_df\
,cpi_vehicles_df,cpi_eggs_df,cpi_aparrel_df,cpi_city_housing_df,real_pce_durable_df,cpi_urban_transportation_df\
pce_df,labor_particip_df,real_PCE_df,pce_durable_df,job_openings_df,pce_nondurable_df,real_disp_income_df]

In [397]:
misc_daily_df.dtypes

Inf_Expectation_5yr                        object
Price_per_Barrel                           object
Price_per_Barrel_Change                   float64
Price_per_Barrel_Pct_Change               float64
Economic_Uncertainty                       object
Economic_Uncertainty_Change               float64
Economic_Uncertainty_Pct_Change           float64
10_Year_Breakeven_Inflation                object
10_Year_Breakeven_Inflation_Change        float64
10_Year_Breakeven_Inflation_Pct_Change    float64
dtype: object

In [402]:
personal_savings_df

,PPI_Metals,PPI_Metals_Change,PPI_Metals_Pct_Change
Date,,,
1959-01-01,39.6,0.0,0.000000
1959-02-01,37.5,-2.1,-5.303030
1959-03-01,36.5,-1.0,-2.666667
1959-04-01,40.0,3.5,9.589041
1959-05-01,38.1,-1.9,-4.750000
...,...,...,...
2020-10-01,2280.3,-186.4,-7.556655
2020-11-01,2136.4,-143.9,-6.310573
2020-12-01,2325.3,188.9,8.841977


In [403]:
pers_savings_df

,Personal_Savings,Personal_Savings_Change,Personal_Savings_Pct_Change
Date,,,
1959-01-01,11.3,0.0,0.000000
1959-02-01,10.6,-0.7,-6.194690
1959-03-01,10.3,-0.3,-2.830189
1959-04-01,11.2,0.9,8.737864
1959-05-01,10.6,-0.6,-5.357143
...,...,...,...
2020-10-01,13.1,-1.0,-7.092199
2020-11-01,12.5,-0.6,-4.580153
2020-12-01,13.5,1.0,8.000000


In [406]:
series_json

{'realtime_start': '2021-04-22',
 'realtime_end': '2021-04-22',
 'observation_start': '1600-01-01',
 'observation_end': '9999-12-31',
 'units': 'lin',
 'output_type': 1,
 'file_type': 'json',
 'order_by': 'observation_date',
 'sort_order': 'asc',
 'count': 746,
 'offset': 0,
 'limit': 100000,
 'observations': [{'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-01-01',
   'value': '175818'},
  {'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-02-01',
   'value': '176044'},
  {'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-03-01',
   'value': '176274'},
  {'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-04-01',
   'value': '176503'},
  {'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-05-01',
   'value': '176723'},
  {'realtime_start': '2021-04-22',
   'realtime_end': '2021-04-22',
   'date': '1959-06-01',
   'value': '1769